# Final Project: Functional Programming

**Laura Belizón Merchán** - 100452273  
**Jorge Lázaro Ruiz** - 100452172  
*Degree in Applied Mathematics and Computation*

## Brief summary

*Pending...*

In [1]:
-- Necessary imports
import Data.Maybe (listToMaybe, fromJust)

## Types

> 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data, newtype) for each of them.

An agenda is described as a list of events, so it makes sense to center the type definitions around the characteristics of an event.

For this, we can create the different types of descriptors defined and then use them for the constructor of an event.

In [2]:
type Name = String
data Type = Personal | Health | Work | Management deriving Show
data Date = Date {day :: Int, month :: String, year :: Int, workday :: Bool} deriving Show
data Time = Time {start :: (Int, Int), end :: (Int, Int), duration :: Maybe Int} deriving Show
newtype Participants = Participants [Name] deriving Show
data Repetitions = Punctual | Daily Int | Weekday Int | Weekly Int deriving Show

* `Name` is just an alias for a `String`, as there are no spectial requirements for it.
* `Type` and `Repetitions` need to be instances of class `Show` if we want to be able to print them, and since they can only take a set of values we must define them as a `data`.
* `Date` and `Time` require a constructor with multiple parameters, hence we use `data` to define them. They also have to be instances of class `Show` so we can pretty-print them.
* `Participants` could be defined as an alias for `[Name]` but in order to avoid confusion with just any other list of strings, we decided to make it into a `newtype` to distinguish it and prevent unintended operations with other lists of strings.

In [3]:
data Descriptor =
    NameDescriptor Name
    | TypeDescriptor Type
    | DateDescriptor Date
    | TimeDescriptor Time
    | ParticipantsDescriptor Participants
    | RepetitionsDescriptor Repetitions
    deriving Show

type Event = [Descriptor]

With all the types defined, all we need to do is create the type `Descriptor`, with a constructor for each field.  
After that, we can define `Event` as a list of `Descriptor`s.

## Event display

> 2. A function to show the events in a pretty way, following the order of descriptors as presented above.

As instances of `Show`, let's define the `show` function for each of the types.

In [15]:
instance Show Type where
    show Personal = "Personal"
    show Health = "Health"
    show Work = "Work"
    show Management = "Management"

instance Show Date where
    show (Date day month year workday) = show day ++ " " ++ month ++ " " ++ show year ++ " " ++
        if workday then "(Workday)" else ""

leadingZero :: Int -> String
leadingZero n = if n < 10 then "0" ++ show n else show n

instance Show Time where
    show (Time (startHour, startMinute) (endHour, endMinute) duration) = 
        show startHour ++ ":" ++ leadingZero startMinute ++ " - " ++ 
        show endHour ++ ":" ++ leadingZero endMinute ++ " " ++ 
        maybe "" ((" " ++) . show) duration

instance Show Participants where
    show (Participants names) = foldl (\acc name -> acc ++ name ++ ", ") "" names

instance Show Repetitions where
    show Punctual = "Punctual"
    show (Daily n) = "Daily " ++ show n
    show (Weekday n) = "Weekday " ++ show n
    show (Weekly n) = "Weekly " ++ show n

Because the `Event` type is just a list of `Descriptor`s with no particular order, we need to sort it before printing it.

For this, we created a set of "getter" functions that finds instances of a given descriptor in a list of descriptors.

In [5]:
import Data.List (find)

getName :: Event -> Maybe Name
getName event = case find isNameDescriptor event of
        Just (NameDescriptor name) -> Just name
        _ -> Nothing
    where
        isNameDescriptor (NameDescriptor _) = True
        isNameDescriptor _ = False

-- Similar changes for the other functions

getType :: Event -> Maybe Type
getType event = case find isTypeDescriptor event of
        Just (TypeDescriptor type') -> Just type'
        _ -> Nothing
    where
        isTypeDescriptor (TypeDescriptor _) = True
        isTypeDescriptor _ = False

getDate :: Event -> Maybe Date
getDate event = case find isDateDescriptor event of
        Just (DateDescriptor date) -> Just date
        _ -> Nothing
    where
        isDateDescriptor (DateDescriptor _) = True
        isDateDescriptor _ = False

getTime :: Event -> Maybe Time
getTime event = case find isTimeDescriptor event of
        Just (TimeDescriptor time) -> Just time
        _ -> Nothing
    where
        isTimeDescriptor (TimeDescriptor _) = True
        isTimeDescriptor _ = False

getParticipants :: Event -> Maybe Participants
getParticipants event = case find isParticipantsDescriptor event of
        Just (ParticipantsDescriptor participants) -> Just participants
        _ -> Nothing
    where
        isParticipantsDescriptor (ParticipantsDescriptor _) = True
        isParticipantsDescriptor _ = False

getRepetitions :: Event -> Maybe Repetitions
getRepetitions event = case find isRepetitionsDescriptor event of
        Just (RepetitionsDescriptor repetitions) -> Just repetitions
        _ -> Nothing
    where
        isRepetitionsDescriptor (RepetitionsDescriptor _) = True
        isRepetitionsDescriptor _ = False

In [13]:
printEvent :: Event -> IO ()
printEvent event = do
    putStrLn "Event:"
    putStrLn $ "  Name: " ++ maybe "Unknown" show (getName event)
    putStrLn $ "  Type: " ++ maybe "Unknown" show (getType event)
    putStrLn $ "  Date: " ++ maybe "Unknown" show (getDate event)
    putStrLn $ "  Time: " ++ maybe "Unknown" show (getTime event)
    putStrLn $ "  Participants: " ++ maybe "Unknown" show (getParticipants event)
    putStrLn $ "  Repetitions: " ++ maybe "Unknown" show (getRepetitions event)

-- Example usage
eventName = NameDescriptor "Example"
eventType = TypeDescriptor Work
eventDate = DateDescriptor (Date 6 "December" 2023 True)
eventTime = TimeDescriptor (Time (10, 30) (12, 0) (Just 90))
eventParticipants = ParticipantsDescriptor (Participants ["John", "Alice", "Bob"])
eventRepetitions = RepetitionsDescriptor (Weekly 2)
exampleEvent = [eventName, eventType, eventDate, eventTime, eventParticipants, eventRepetitions]
printEvent exampleEvent

Event:
  Name: "Example"
  Type: Work
  Date: 6 December 2023 True
  Time: 10:30 - 12:00  90
  Participants: John, Alice, Bob, 
  Repetitions: Weekly 2

In [7]:
printEvent event = do
    putStrLn "Event:"
    putStrLn $ "  Name: " ++ show (fromJust $ getName event)
    putStrLn $ "  Type: " ++ show (fromJust $ getType event)
    putStrLn $ "  Date: " ++ show (fromJust $ getDate event)
    putStrLn $ "  Time: " ++ show (fromJust $ getTime event)
    putStrLn $ "  Participants: " ++ show (fromJust $ getParticipants event)
    putStrLn $ "  Repetitions: " ++ show (fromJust $ getRepetitions event)

In [8]:
printEvent event = do
    putStrLn "Event:"
    putStrLn $ "  Name: " ++ show (fromJust $ getName event)
    putStrLn $ "  Type: " ++ show (fromJust $ getType event)
    putStrLn $ "  Date: " ++ show (fromJust $ getDate event)
    putStrLn $ "  Time: " ++ show (fromJust $ getTime event)
    putStrLn $ "  Participants: " ++ show (fromJust $ getParticipants event)
    putStrLn $ "  Repetitions: " ++ show (fromJust $ getRepetitions event)